In [40]:
import os
import json
import pandas as pd
from langchain import hub
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.tools import BaseTool, StructuredTool, tool
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.agents import AgentExecutor, create_react_agent, create_tool_calling_agent

load_dotenv('../.env')


True

In [41]:
TABLE_NAME = os.getenv('EFORM_TABLE_NAME')
DB_NAME = os.getenv('EFORM_DB_NAME')
USER = "kamalleaf" #os.getenv('EFORM_DB_USER')
PASS = os.getenv('EFORM_DB_PASS')
HOST = os.getenv('EFORM_DB_HOST')
GPT_4 = "gpt-4-1106-preview"
GPT_3 = "gpt-3.5-turbo-0125"
LLM = ChatOpenAI(temperature=0, model=GPT_4)
PROMPT = hub.pull("hwchase17/react")


In [42]:
SESSION_ID="test"

In [43]:
prompt = hub.pull("hwchase17/openai-functions-agent")

In [44]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [45]:
message_history = ChatMessageHistory()

In [46]:
agent = create_tool_calling_agent(LLM, [multiply], prompt)

In [48]:
agent_executor = AgentExecutor(agent=agent, tools=[multiply], verbose=True)

In [49]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [50]:
agent_with_chat_history.invoke(
    {"input": "hi! I'm bob"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": SESSION_ID}},
)

Parent run 3e5a5b53-3acc-4edc-ad31-8003624fbf26 not found for run dbb39179-4370-4556-a129-912a8e3aba79. Treating as a root run.




> Entering new AgentExecutor chain...
Hello Bob! How can I assist you today?

> Finished chain.


{'input': "hi! I'm bob",
 'chat_history': [],
 'output': 'Hello Bob! How can I assist you today?'}

In [51]:
agent_with_chat_history.invoke(
    {"input": "whats my name again?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": SESSION_ID}},
)

Parent run 6fb767fc-8b74-47cc-bcb5-41997db7eab8 not found for run 26e762e1-b690-44e7-bcc0-ffd2cd562bb1. Treating as a root run.




> Entering new AgentExecutor chain...
Your name is Bob. How can I help you further, Bob?

> Finished chain.


{'input': 'whats my name again?',
 'chat_history': [HumanMessage(content="hi! I'm bob"),
  AIMessage(content='Hello Bob! How can I assist you today?')],
 'output': 'Your name is Bob. How can I help you further, Bob?'}